# 20146703 박지호

## Assignment 7

input으로 image를 하나 선택, k-means algorithm을 적용. 클러스터의 수와 intensity energy와 spatial energy 사이의 trade-off 고려

object function은 다음과 같다

$$ \sum_k \sum_\{ x \in I(k) \} [ \| f(x) - m_k \|^2 + a * \| x - c_k \|^2 ] $$

l(k): cluster k에 속하는 x의 index set

m_k: cluster k에 대한 image intensity의 centroid

c_k: cluster k에 대한 spatial location의 centroid

a: image intensity와 spatial location 사이의 중요성을 결정

1. 클러스터링된 결과를 visualization하기 (k를 다양하게 해보기, 클러스터마다 centroid color를 이용해서

2. intensity energy graph와 spatial energy graph를 visualization하기
